In [2]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = '/home/data/train.json'

# Read the JSON file - it's a JSON array, not line-delimited
with open(train_path, 'r') as f:
    train_data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(train_data)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Dataset shape: (2878, 32)
Columns: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request', 'requester_upvotes_minus_down

In [3]:
# Check target distribution
print("Target distribution:")
print(df['requester_received_pizza'].value_counts())
print(f"\nPercentage distribution:")
print(df['requester_received_pizza'].value_counts(normalize=True) * 100)

# Check data types
print(f"\nData types:")
print(df.dtypes.head(10))

Target distribution:
requester_received_pizza
False    2163
True      715
Name: count, dtype: int64

Percentage distribution:
requester_received_pizza
False    75.156359
True     24.843641
Name: proportion, dtype: float64

Data types:
giver_username_if_known                         object
number_of_downvotes_of_request_at_retrieval      int64
number_of_upvotes_of_request_at_retrieval        int64
post_was_edited                                 object
request_id                                      object
request_number_of_comments_at_retrieval          int64
request_text                                    object
request_text_edit_aware                         object
request_title                                   object
requester_account_age_in_days_at_request       float64
dtype: object


In [4]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum().sort_values(ascending=False).head(15))

# Check text length statistics
print("\nText length statistics:")
df['request_text_length'] = df['request_text'].str.len()
df['request_title_length'] = df['request_title'].str.len()
print(df['request_text_length'].describe())
print(f"\nTitle length stats:")
print(df['request_title_length'].describe())

Missing values:
requester_user_flair                                    2163
giver_username_if_known                                    0
number_of_upvotes_of_request_at_retrieval                  0
number_of_downvotes_of_request_at_retrieval                0
request_id                                                 0
request_number_of_comments_at_retrieval                    0
request_text                                               0
post_was_edited                                            0
request_title                                              0
requester_account_age_in_days_at_request                   0
requester_account_age_in_days_at_retrieval                 0
requester_days_since_first_post_on_raop_at_request         0
requester_days_since_first_post_on_raop_at_retrieval       0
requester_number_of_comments_at_request                    0
requester_number_of_comments_at_retrieval                  0
dtype: int64

Text length statistics:
count    2878.000000
mean      

In [5]:
# Check user flair distribution
print("User flair distribution:")
print(df['requester_user_flair'].value_counts(dropna=False))

# Check post_was_edited distribution
print("\nPost edited distribution:")
print(df['post_was_edited'].value_counts())

# Check numerical features correlation with target
numerical_features = ['number_of_upvotes_of_request_at_retrieval', 
                      'number_of_downvotes_of_request_at_retrieval',
                      'request_number_of_comments_at_retrieval',
                      'requester_account_age_in_days_at_request',
                      'requester_number_of_comments_at_request',
                      'requester_number_of_posts_at_request']

print("\nCorrelation with target:")
for feature in numerical_features:
    corr = df[feature].corr(df['requester_received_pizza'])
    print(f"{feature}: {corr:.4f}")

User flair distribution:
requester_user_flair
None      2163
shroom     677
PIF         38
Name: count, dtype: int64

Post edited distribution:
post_was_edited
False           2423
True             241
1375324604.0       1
1366314331.0       1
1367280954.0       1
                ... 
1379372126.0       1
1378425306.0       1
1374109637.0       1
1358627245.0       1
1372729287.0       1
Name: count, Length: 216, dtype: int64

Correlation with target:
number_of_upvotes_of_request_at_retrieval: 0.0908
number_of_downvotes_of_request_at_retrieval: 0.0206
request_number_of_comments_at_retrieval: 0.2907
requester_account_age_in_days_at_request: 0.0434
requester_number_of_comments_at_request: 0.0367
requester_number_of_posts_at_request: 0.0380


In [6]:
# Fix the post_was_edited issue - it contains timestamps instead of booleans
print("Unique values in post_was_edited (first 10):")
print(df['post_was_edited'].unique()[:10])

# Create a proper boolean column
df['post_was_edited_clean'] = df['post_was_edited'].apply(lambda x: str(x).lower() == 'true')
print(f"\nCleaned post_was_edited distribution:")
print(df['post_was_edited_clean'].value_counts())

# Check correlation with target
print(f"\nCorrelation of edited posts with receiving pizza: {df['post_was_edited_clean'].corr(df['requester_received_pizza']):.4f}")

# Check user flair correlation
flair_dummies = pd.get_dummies(df['requester_user_flair'], dummy_na=True)
for col in flair_dummies.columns:
    corr = flair_dummies[col].corr(df['requester_received_pizza'])
    print(f"Flair {col} correlation: {corr:.4f}")

Unique values in post_was_edited (first 10):
[False True 1363315140.0 1349998232.0 1364413726.0 1342991229.0
 1351709165.0 1347224731.0 1347230374.0 1371296113.0]

Cleaned post_was_edited distribution:
post_was_edited_clean
False    2637
True      241
Name: count, dtype: int64

Correlation of edited posts with receiving pizza: 0.1252
Flair PIF correlation: 0.2012
Flair shroom correlation: 0.9646
Flair nan correlation: -1.0000
